In [3]:
import json
import pickle

import numpy as np
from numpy import random
import random as rn
random.seed(12345)
import tensorflow as tf
np.random.seed(12345)
rn.seed(12345)
from underthesea import word_tokenize,  pos_tag
import os, pickle, re, keras, sklearn, string
from pyvi import ViTokenizer, ViPosTagger
from transformers import AutoModel, AutoTokenizer
from keras.preprocessing.sequence import pad_sequences
from scipy import spatial
import string
with open("sorted.json","rb") as h:
    read_data=json.load(h)
with open("set_addressdict_official.pickle","rb") as h:
    address_pickle_set=pickle.load(h) 
    

In [2]:
def normalText(sent):
    sent = str(sent).replace('_',' ').replace('/',' trên ')
    sent = re.sub('-{2,}','',sent)
    sent = re.sub('\\s+',' ', sent)
    patPrice = r'([0-9]+k?(\s?-\s?)[0-9]+\s?(k|K))|([0-9]+(.|,)?[0-9]+\s?(triệu|ngàn|trăm|k|K|))|([0-9]+(.[0-9]+)?Ä‘)|([0-9]+k)'
    patHagTag = r'#\s?[aăâbcdđeêghiklmnoôơpqrstuưvxyàằầbcdđèềghìklmnòồờpqrstùừvxỳáắấbcdđéếghíklmnóốớpqrstúứvxýảẳẩbcdđẻểghỉklmnỏổởpqrstủửvxỷạặậbcdđẹệghịklmnọộợpqrstụựvxỵãẵẫbcdđẽễghĩklmnõỗỡpqrstũữvxỹAĂÂBCDĐEÊGHIKLMNOÔƠPQRSTUƯVXYÀẰẦBCDĐÈỀGHÌKLMNÒỒỜPQRSTÙỪVXỲÁẮẤBCDĐÉẾGHÍKLMNÓỐỚPQRSTÚỨVXÝẠẶẬBCDĐẸỆGHỊKLMNỌỘỢPQRSTỤỰVXỴẢẲẨBCDĐẺỂGHỈKLMNỎỔỞPQRSTỦỬVXỶÃẴẪBCDĐẼỄGHĨKLMNÕỖỠPQRSTŨỮVXỸ]+'
    patURL = r"(?:http://|www.)[^\"]+"
    sent = re.sub(patURL,'website',sent)
    sent = re.sub(patHagTag,' hagtag ',sent)
    sent = re.sub(patPrice, ' giá_tiền ', sent)
    sent = re.sub('\.+','.',sent)
    sent = re.sub('(hagtag\\s+)+',' hagtag ',sent)
    sent = re.sub('\\s+',' ',sent)
    return sent


def normalize_elonge_word(sent):
    s_new = ''
    for word in sent.split(' '):
        word_new = ''
        for char in word.strip():
            if char != word_new[-1]:
                word_new += char
    s_new += word_new.strip() + ' '
    return s_new

def tokenizer(text):
      token = word_tokenize(text,format="text")
      token = token.replace('giá tiền','giá_tiền').replace('Giá tiền','Giá_tiền')
      return token

def deleteIcon(text):
    text = text.lower()
    s = ''
    pattern = r"[a-zA-ZaăâbcdđeêghiklmnoôơpqrstuưvxyàằầbcdđèềghìklmnòồờpqrstùừvxỳáắấbcdđéếghíklmnóốớpqrstúứvxýảẳẩbcdđẻểghỉklmnỏổởpqrstủửvxỷạặậbcdđẹệghịklmnọộợpqrstụựvxỵãẵẫbcdđẽễghĩklmnõỗỡpqrstũữvxỹAĂÂBCDĐEÊGHIKLMNOÔƠPQRSTUƯVXYÀẰẦBCDĐÈỀGHÌKLMNÒỒỜPQRSTÙỪVXỲÁẮẤBCDĐÉẾGHÍKLMNÓỐỚPQRSTÚỨVXÝẠẶẬBCDĐẸỆGHỊKLMNỌỘỢPQRSTỤỰVXỴẢẲẨBCDĐẺỂGHỈKLMNỎỔỞPQRSTỦỬVXỶÃẴẪBCDĐẼỄGHĨKLMNÕỖỠPQRSTŨỮVXỸ,._]"
    
    for char in text:
        if char !=' ':
            if len(re.findall(pattern, char)) != 0:
                s+=char
            elif char == '_':
                s+=char
        else:
            s+=char
    s = re.sub('\\s+',' ',s)
    return s.strip()

def normalize_elonge_word(sent):
    s_new = ''
    for word in sent.split(' '):
        word_new = ' '
        for char in word.strip():
            if char != word_new[-1]:
                word_new += char
        s_new += word_new.strip() + ' '
    return s_new.strip()  

def clean_doc(doc,set_work=False):
    for punc in string.punctuation:
        doc = doc.replace(punc,' '+ punc + ' ')
    doc = normalText(doc)
    doc = deleteIcon(doc)
    if set_work:
      doc = word_tokenize(doc,format="text")
    # Lowercase
    doc = doc.lower()
    # Removing multiple whitespaces
    doc = re.sub(r"\?", " \? ", doc)
    # Remove numbers
    doc = re.sub(r"[0-9]+", " num ", doc)
    # Split in tokens
    # Remove punctuation
    for punc in string.punctuation:
      if punc not in "_":
          doc = doc.replace(punc,' ')
    doc = re.sub('\\s+',' ',doc)
    doc = normalize_elonge_word(doc)
    return doc

In [ ]:
#https://gist.github.com/J2TEAM/9992744f15187ba51d46aecab21fd469

In [4]:
def extract_elemts(h_dict):
    return h_dict[2]+" "+h_dict[1]
address=[]
for province in read_data:
    text_province=extract_elemts(province)
    for ward in province[4]:
        text_ward=extract_elemts(ward)
        for thi_tran in ward[4]:
            text_thi_tran=extract_elemts(thi_tran)
            full_addresses=f'{text_thi_tran}, {text_ward},{text_province}'
            address.append(full_addresses)
address_from_Foody=[]
for h in address_pickle_set:
    for k in list(address_pickle_set[h]):
        address.append(k.replace(", \n","").replace("\n",""))
address_=[clean_doc(text,True) for text in address]


In [57]:
alternatives={'phường':['ph','p','phuong'],
'quận':['quan','q'],
'thành_phố':['thanh_pho','tp','t p'],
'tp':['thanh_pho','tp','t p'],
'thị_xã':['thi_xa','tx','t x'],
'thị_trấn':['tt','t','thi_tran'],
'tt':['thị_trấn','t','thi_tran'],
'p':['phường','phuong'],
'xã':['xa','x'],
'cao_đẳng':['cao_dang','cd','c d'],
'câu_lạc_bộ': ['cau_lac_bo','clb','c l b'],
'công_ty cổ_phần':['cong_ty co_phan','ctcp','c t c p','cty cp','cty c p'],
'công ty':['cong_ty','cty'],
'công_viên văn_hóa':['cong_vien van_hoa','cvvh','c v v h'],
'công_viên':['cong_vien','cv','c v'],
'đại_học':['dai_hoc','dh','d h'],
'đường sắt':['duong sat','ds','d s'],
'khu_công_nghiệp':['khu_cong_nghiep','kcn','k c n'],
'khu du_lịch':['khu du_lich','kdl','k d l'],
'khu nghỉ_mát':['khu nghi_mat','knm','k n m'],
'khách_sạn':['khach_san','ks','k / s'],
'nhà_hát': ['nha hat','nh','n h'],
'nhà thi_đấu':['nha thi_dau','ntd','n t d'],
'nhà_thờ':['nha tho','nt','n t'],
'rạp hát':['rap hat','rh','r h'],
'sân_bay quốc_tế':['san_bay quoc_te','sbqt','s b q t'],
'sân_bay':['san bay','sb','s b'],
'sân vận_động':['san van dong','svd','s v d'],
'tiểu_học':['tieu_hoc','th','t h'],
"trung_học_cơ_sở":['trung_hoc_co_so','thcs','t h c s'],
"trung_học_phổ_thông":['trung_hoc_pho_thong','thpt','t h p t'],
"trung_tâm thương_mại":['trung_tam thuong_mai','tttm','t t t m'],
"mall":["trung_tâm thương_mại",'trung_tam thuong_mai','tttm','t t t m'],
'trung_tâm':["trung_tam",'tt','t t'],
'tổng công_ty':['tong cong_ty','tcty','tct'],
'viện_bảo_tàng':['vien_bao_tang','vbt','v b t'],
'vươn quốc_gia':['vuon quoc_gia','vqg','v q g'],
'công_trường':['cong_truong','ct','c t'],
'đại_lộ':['dai_lo','đl','d l'],
'đường':['duong','d'],
'đường hẻm':['duong hem','hẻm'],
'đường nhỏ':['duong nho'],
'đường_phố':['duong pho'],
'quảng_trường':['quang truong','qt','q t'],
'quốc_lộ':['quoc lo','ql','q l'],
'tỉnh lộ':['tinh lo','tl','t l'],
'Hotel':['khach_san','Khách_san','Khách_sạn']
}

In [55]:
word_tokenize("tổng công ty",format="text")

'tổng công_ty'

In [106]:
##removing accents
s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
def remove_accents(input_str):
	s = ''
	for c in input_str:
		if c in s1:
			s += s0[s1.index(c)]
		else:
			s += c
	return s

from tqdm import tqdm
for index,text in tqdm(enumerate(address_)):
    text_split=text.split(" ")
    for sub_text in text_split:
        if sub_text in alternatives.keys():
            for change_phrase in list(alternatives[sub_text]):
                new_augment_Text=text.replace(sub_text,change_phrase)
                print(new_augment_Text+"\n")
                address_.append(new_augment_Text)
                address_.append(remove_accents(new_augment_Text))

0it [00:00, ?it/s]


KeyError: 'sub_text'